# Workshop Preperation for Classiq's Challenge - Quantum Arithmetics

Welcome to the Classiq challenge of the 2024 MIT IQuHack hackathon!
This Jupyter notebook is a tutorial workshop that should prepare you for the challenge itself.


Additional resources you should use are
- The IDE of the classiq platform at [platform.classiq.io](platform.classiq.io)
- The [community Slack of Classiq](https://short.classiq.io/join-slack) - Classiq's team will answer any question you have over there, including implementation questions
- [Classiq's documentation](https://docs.classiq.io/latest/user-guide/platform/) with the dedicated [Python SDK explanations](https://docs.classiq.io/latest/user-guide/platform/qmod/python/functions/)

Good luck!

![Classiq](MITClassiq.png)

## Setting The Scene

Install the Classiq SDK package:

In [3]:
%pip install -U classiq

  Using cached classiq-0.36.1-py3-none-any.whl (364 kB)
  Using cached ConfigArgParse-1.7-py3-none-any.whl (25 kB)
  Using cached Pyomo-6.5.0-cp311-cp311-win_amd64.whl (4.0 MB)
  Using cached httpx-0.26.0-py3-none-any.whl (75 kB)
  Using cached keyring-23.13.1-py3-none-any.whl (37 kB)
  Using cached networkx-2.8.8-py3-none-any.whl (2.0 MB)
  Using cached numexpr-2.9.0-cp311-cp311-win_amd64.whl (96 kB)
  Using cached packaging-21.3-py3-none-any.whl (40 kB)
  Using cached pandas-1.5.3-cp311-cp311-win_amd64.whl (10.3 MB)
  Using cached plotly-5.18.0-py3-none-any.whl (15.6 MB)
  Using cached pydantic-1.10.14-cp311-cp311-win_amd64.whl (2.1 MB)
  Using cached sympy-1.10.1-py3-none-any.whl (6.4 MB)
  Using cached anyio-4.2.0-py3-none-any.whl (85 kB)
  Using cached httpcore-1.0.2-py3-none-any.whl (76 kB)
  Using cached jaraco.classes-3.3.0-py3-none-any.whl (5.9 kB)
  Using cached importlib_metadata-7.0.1-py3-none-any.whl (23 kB)
  Using cached pywin32_ctypes-0.2.2-py3-none-any.whl (30 kB)
    

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
perceval-quandela 0.10.3 requires networkx~=3.1, but you have networkx 2.8.8 which is incompatible.
black 24.1.1 requires packaging>=22.0, but you have packaging 21.3 which is incompatible.
pyproject-api 1.6.1 requires packaging>=23.1, but you have packaging 21.3 which is incompatible.
tox 4.12.1 requires packaging>=23.2, but you have packaging 21.3 which is 

You need to authenticate your device in order to use Classiq's backend synthesis engine and IDE. Make sure to register to the platform at [platform.classiq.io](platform.classiq.io) before you run the next cell:

In [4]:
import classiq
classiq.authenticate()

Your user code: WMTG-GBZF
If a browser doesn't automatically open, please visit the url: https://auth.classiq.io/activate?user_code=WMTG-GBZF


In [5]:
from classiq import *

## A Warm Up

### First Example

Write a function that prepares the minus state $\ket{-}=\frac{1}{\sqrt2}(\ket{0}-\ket{1})$, assuming it recives the qubit $\ket{x}=\ket{0}$ (hint: 

<details>
<summary>
HINT
</summary>

Use `H(x)`,`X(x)`
</details>

In [6]:
@QFunc
def prepare_minus_state(x:QBit):
    X(x)
    H(x)

Now we will test our code:

In [7]:
@QFunc
def main(x: Output[QBit]):
    allocate(1,x) # Initalize the qubit x
    prepare_minus_state(x)

In [4]:
quantum_model = create_model(main)
quantum_program = synthesize(quantum_model)

In [5]:
show(quantum_program)

Opening: https://platform.classiq.io/circuit/00a0d989-1d56-431a-9c21-f4e33273df89?version=0.36.1


Some basic explanations about the high-level functional design with Classiq:

* There should always be a main (`def main(...)`) function - the model that captures your algortihm is described there

* The model is always generated out of the main function 

* The model is sent to the synthesis engine (compiler) that return a quantum program which contains the quantum circuit


Some basic guidelines about the modeling language (QMOD):

1. Every quantum variable should be declared, either as a parameter of a funciton e.g. `def prepare_minus(x: QBit)` or within the function itself with `x = QBit('x')`

2. Some quantum variables need to be initalized with the `allocate` function. This is required in 2 cases:
* A variable is a parameter of a function with the declaration `Output` like `def main(x: Output[QInt])`
* A variable that was declared within a function like `a = QInt('a')`

3. For the `main` function, you will always use `Output` for all variables, as the function does not receive any input

4. Every function you use with the QMOD language should have the decorator `@QFunc` before it

Important tip!

You can see all the declarations of the functions with what are their input arguments in the `functions.py` file within the classiq package (or by just right clicking a function and presing `Go To Defintion`)

### Uniform Superposition

Let's continue warming up with creating a function that receives a quantum register and creates a uniform superposistion for all qubits within this array. You should use the function `apply_to_all(gate_operand=, target=)`:

In [8]:
@QFunc
def create_initial_state(reg: QArray[QBit]):
    apply_to_all(H,reg)

Test yout function by creating a new main function, synthesizing and viewing the circuit:

In [9]:
@QFunc
def main(reg: Output[QArray]): #TODO fill int the correct declaration here, what variables this model shoul output?
    allocate(4,reg)
    create_initial_state(reg)

In [10]:
qprog = synthesize(create_model(main))
show(qprog)

Opening: https://platform.classiq.io/circuit/e95c1005-8ffd-425a-8930-ae38cb582521?version=0.36.1


## Arithmetic Operations with Classiq

One of the key advantages of Classiq is it's simplistic and powerful compiler for quantum arithmetics. Let's see an example:

In [11]:
num_qubits = 4
fraction_digits = 0 
is_signed = True

@QFunc
def main(x: Output[QNum], y: Output[QNum]):
    allocate_num(num_qubits=num_qubits, is_signed=is_signed, fraction_digits=fraction_digits, out=x)
    hadamard_transform(x)
    y|= x**2 + 1

qmod = create_model(main)

The `allocate_num` function initalizes a quantum variable that represent numbers. By default it is initalized to the $\ket{0}$ state. Then the `hadmard_transform` create a superposition of all posible states in the domain $[-2^3,2^3-1]$. Finally, the arithmetic operation creates the entangled superpostion of states:
$\begin{equation}
\sum_{x =-2^3}^{2^3-1}\ket{x}\ket{x^2+1}
\end{equation}$

The `qmod` variable is a text file that captures the algortihm we have just created. Now, what we want is to synthesize (compile) in order to receive a concrete quantum program that contains the quantum circuit implementation.

In [12]:
qprog = synthesize(qmod)

And in order to view it:

In [13]:
show(qprog)

Opening: https://platform.classiq.io/circuit/e18a945d-7490-4008-a704-c3b82111dd07?version=0.36.1


## Advanced Arithmetics

Now let's create a general linear function with Classiq: $y= ax+b$ where $a,b$ are classical integer parameters and $x,y$ is a quantum states representing integers:

In [14]:
@QFunc
def linear_func(a:QParam[int],b: QParam[int], x:QNum, y: Output[QNum]):
    y |= a*x+b

In [15]:
@QFunc
def main(x:Output[QNum], y: Output[QNum]):

    a = 2
    b = 1
    allocate_num(num_qubits=4,is_signed=False,fraction_digits=0,out=x)
    hadamard_transform(x)
    linear_func(a,b,x,y)

qmod = create_model(main)

In [16]:
qprog = synthesize(qmod)

Let's execute the circuit from directly from the SDK:

In [17]:
job = execute(qprog)

And we can view the results in the IDE:

In [18]:
job.open_in_ide()

Or to directly analyze it within the SDK:

In [14]:
results = job.result()
parsed_counts = results[0].value.parsed_counts
for sampled_state in parsed_counts: print(sampled_state.state)

{'x': 5.0, 'y': 11.0}
{'x': 3.0, 'y': 7.0}
{'x': 6.0, 'y': 13.0}
{'x': 9.0, 'y': 19.0}
{'x': 0.0, 'y': 1.0}
{'x': 12.0, 'y': 25.0}
{'x': 11.0, 'y': 23.0}
{'x': 4.0, 'y': 9.0}
{'x': 15.0, 'y': 31.0}
{'x': 8.0, 'y': 17.0}
{'x': 14.0, 'y': 29.0}
{'x': 1.0, 'y': 3.0}
{'x': 13.0, 'y': 27.0}
{'x': 2.0, 'y': 5.0}
{'x': 10.0, 'y': 21.0}
{'x': 7.0, 'y': 15.0}


Now it's your turn! Implement the same linear function, but now $x$ is in the domain $[0,1)$ and is represented by 4 qubits. The parameters $a,b$ should be now `float` with the values of: $a=0.5, b=1.5$.

In [11]:
@QFunc
def linear_func(a:QParam[float],b: QParam[float], x:QNum, y: Output[QNum]):
    y |= a*x+b

@QFunc
def main(x:Output[QNum], y: Output[QNum]):

    a = 0.5
    b = 1.5
    allocate_num(num_qubits=4,is_signed=False,fraction_digits=4,out=x)
    hadamard_transform(x)
    linear_func(a,b,x,y)

qmod = create_model(main)

In [12]:
qprog = synthesize(qmod)

## Tutorial - Two controlled Linear operations

Let's say we want now to have two linear operations applied on the same quantum variable (register). But the arithmetic operation initalize a new quantum variable, so how can we do that? The answer is that we need to apply the operation to another variable and then XOR it to the variable we want. 

This can be useful if the linear operation we want to apply is controlled upon a control variable. Let's first define the functional buildng block:

In [16]:
@QFunc
def inplace_linear_attempt(a:QParam[int],b: QParam[int], x:QNum, y: QNum):
    tmp = QNum('tmp')
    linear_func(a,b,x,tmp)
    inplace_xor(tmp,y)

And checking our basic function implementation:

In [17]:
@QFunc
def main(x: Output[QNum],y: Output[QNum]):
    a = 1
    b = 2

    allocate_num(4,False,0,y)
    allocate_num(4,False,0,x)
    hadamard_transform(x)
    inplace_linear_attempt(a,b,x,y)

qmod = create_model(main)
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/01aff113-35aa-4c38-9a25-8817b4eed7ee?version=0.36.1


Ok, cool. So now we want to add a control qubit that controlled on the state $\ket{0}$ implements the linear funciton $\ket{x}\rightarrow\ket{x}\ket{x+2}$ and controlled on the state $\ket{1}$ implements the linear function $\ket{x}\rightarrow\ket{x}\ket{2x+1}$:

In [18]:
@QFunc
def control_logic(a: QParam[list[int]], b: QParam[list[int]], controller:QNum, x: QNum, y: QNum):
    
    repeat( count=a.len(),         
            iteration=lambda i: quantum_if(controller==i, lambda: inplace_linear_attempt(a[i],b[i],x,y)))


In [19]:
@QFunc
def main(controller: Output[QNum], x: Output[QNum],y: Output[QNum]):

    # Linear polynom parameters
    a = [1,2]
    b = [2,1]

    # Initalizing x to a superposition in the domain [0,2^4-1]
    allocate_num(4,False,0,x)
    hadamard_transform(x)
    
    #Initalize y
    allocate_num(4,False,0,y)

    # Setting the controller in a superpostion
    allocate_num(1,False,0,controller)
    H(controller)

    # Implementing the control logic
    control_logic(a,b,controller,x,y)

    
qmod = create_model(main)

In [20]:
qprog = synthesize(qmod)
show(qprog)

Opening: https://platform.classiq.io/circuit/beeb1275-6543-491f-af1a-abe7caf3b4f0?version=0.36.1


By executing we can actually see we get what we want:

In [21]:
def print_parsed_counts(job):
    results = job.result()
    parsed_counts = results[0].value.parsed_counts
    for parsed_state in parsed_counts: print(parsed_state.state)

job = execute(qprog)
print_parsed_counts(job)

{'controller': 0.0, 'x': 3.0, 'y': 5.0}
{'controller': 1.0, 'x': 5.0, 'y': 11.0}
{'controller': 0.0, 'x': 2.0, 'y': 4.0}
{'controller': 1.0, 'x': 8.0, 'y': 1.0}
{'controller': 0.0, 'x': 7.0, 'y': 9.0}
{'controller': 0.0, 'x': 9.0, 'y': 11.0}
{'controller': 0.0, 'x': 13.0, 'y': 15.0}
{'controller': 1.0, 'x': 1.0, 'y': 3.0}
{'controller': 1.0, 'x': 14.0, 'y': 13.0}
{'controller': 1.0, 'x': 0.0, 'y': 1.0}
{'controller': 1.0, 'x': 2.0, 'y': 5.0}
{'controller': 1.0, 'x': 9.0, 'y': 3.0}
{'controller': 0.0, 'x': 14.0, 'y': 0.0}
{'controller': 0.0, 'x': 5.0, 'y': 7.0}
{'controller': 1.0, 'x': 3.0, 'y': 7.0}
{'controller': 0.0, 'x': 10.0, 'y': 12.0}
{'controller': 0.0, 'x': 6.0, 'y': 8.0}
{'controller': 1.0, 'x': 10.0, 'y': 5.0}
{'controller': 0.0, 'x': 1.0, 'y': 3.0}
{'controller': 1.0, 'x': 7.0, 'y': 15.0}
{'controller': 1.0, 'x': 12.0, 'y': 9.0}
{'controller': 1.0, 'x': 4.0, 'y': 9.0}
{'controller': 1.0, 'x': 13.0, 'y': 11.0}
{'controller': 0.0, 'x': 8.0, 'y': 10.0}
{'controller': 0.0, 'x': 

Of course there is the issue of rounding and overflow - when one tries to represent $2*15+1=31$ with $4$ binary digits that's not possible (because the domain $[0,31]$ of integers is represented by at least 5 bits). See our [documentation](https://docs.classiq.io/latest/user-guide/platform/qmod/python/quantum-expressions/#inplace-arithmetic-operators) for further explanations.

Let's try to use Classiq and optimize the circuit for minimal circuit width:

In [22]:
def print_depth_width(quantum_program):
    generated_circuit = GeneratedCircuit.parse_raw(quantum_program)
    print(f"Synthesized circuit width: {generated_circuit.data.width}, depth: {generated_circuit.transpiled_circuit.depth}")

qmod = set_constraints(qmod,Constraints(optimization_parameter='width'))
qprog = synthesize(qmod)
print_depth_width(qprog)

Synthesized circuit width: 20, depth: 688


And when optimizng for depth:

In [23]:
qmod = set_constraints(qmod,Constraints(optimization_parameter='depth'))
qprog = synthesize(qmod)
print_depth_width(qprog)

Synthesized circuit width: 32, depth: 398


Firstly, we can see here a clear demonstration of the power of high-level functional design! The same algortihm with the same functionality was optimized once for depth and once for width and the result is 2 different circuits with different characteristics that implement the same functionality.

Secondly, is this the best we can do? Obviously the Classiq synthesis engine is optimizing for us the algortihm quite good. But, can we change something with our functionality, with our algorithm to get more efficient circuits? 

If we go back to out `inplace_linear_attempt` function, we can see that we initalize a `tmp` variable that requires more qubits and is not used. For such scenarios we have the `within_apply`. This logic implemnts sort of $UVU^{\dagger}$ and when temporary variables are outputs of $U$ and used only by $V$ they are uncomputed by $U^{\dagger}$. Let's see for our example:

In [24]:
@QFunc
def inplace_linear_func(a:QParam[int],b: QParam[int], x:QNum, y: QNum):
    tmp = QNum('tmp')
    within_apply(compute= lambda: linear_func(a,b,x,tmp),
                action= lambda: inplace_xor(tmp,y))

With the new `control_logic`:

In [25]:
@QFunc
def control_logic_2(a: QParam[list[int]], b: QParam[list[int]], controller:QNum, x: QNum, y: QNum):
    
    repeat( count=a.len(),         
            iteration=lambda i: quantum_if(controller==i, lambda: inplace_linear_func(a[i],b[i],x,y)))

And when we put all together now:

In [26]:
@QFunc
def main(controller: Output[QNum], x: Output[QNum],y: Output[QNum]):

    # Linear polynom parameters
    a = [1,2]
    b = [2,1]

    # Initalizing x to a superposition in the domain [0,2^4-1]
    allocate_num(4,False,0,x)
    hadamard_transform(x)
    
    #Initalize y
    allocate_num(4,False,0,y)

    # Setting the controller in a superpostion
    allocate_num(1,False,0,controller)
    H(controller)

    # Implementing the control logic
    control_logic_2(a,b,controller,x,y)

    
qmod = create_model(main)

In [27]:
qprog = synthesize(qmod)

In [34]:
show(qprog)

Opening: https://platform.classiq.io/circuit/c5d18fda-ab8b-42b8-8373-4a57c49283db?version=0.36.1


And now when we optimize for width:

In [28]:
qmod = set_constraints(qmod,Constraints(optimization_parameter='width'))
qprog = synthesize(qmod)
print_depth_width(qprog)

Synthesized circuit width: 16, depth: 244


And for depth:

In [29]:
qmod = set_constraints(qmod,Constraints(optimization_parameter='depth'))
qprog = synthesize(qmod)
print_depth_width(qprog)

Synthesized circuit width: 45, depth: 203


So using the `within_apply` logic enabled us to reduce the optimal circuit implementation in terms of width from $20$ to $16$ and the optimal circuit depth from $398$ to $203$! I think both can be useful for you for the hackahton :) 

## Cheat Sheet

### Initalizations

In [ ]:
allocate(
    num_qubits: QParam[int],
    out: Output[QArray[QBit, Literal["num_qubits"]]])
    '''
    x = QArray('x')
    allocate(4,x)
    '''

allocate_num(
    num_qubits: QParam[int],
    is_signed: QParam[bool],
    fraction_digits: QParam[int],
    out: Output[QNum])
'''
x = QNum('x')
allocate_num(4,False,4,x)
'''

### Operations

In [ ]:
repeat(
    count: QParam[int],
    iteration: QCallable[QParam[int]],
)
'''
x = QArray
allocate(4,x)
repeat(x.len(),lambda index: H(x))
'''

control(
    operand: QCallable,
    ctrl: QArray[QBit],
) 
'''
x = QArray('x')
y = QArray('y')
x = allocate(4,x)
y = allocate(4,y)
repeat(x.len(),lambda i: control(lambda: X(y[i]),x[i]))
'''